In [1]:
# [1] imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
# [2] Create Fully Connected Network
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

'''
# Basic model Checks
model = NN(784, 10)
x = torch.randn(64, 784) #(imagesminibatch, input_size [MNIST 28x28 = 784]) 
print(model(x).shape) 
'''

'\n# Basic model Checks\nmodel = NN(784, 10)\nx = torch.randn(64, 784) #(imagesminibatch, input_size [MNIST 28x28 = 784]) \nprint(model(x).shape) \n'

In [3]:
# [3] Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(device)

In [4]:
# [4] Hyperparameters
INPUT_SIZE = 784
NUM_CLASSES = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 64
NUM_EPOCHS = 8

In [5]:
# [5] Load data
# Training dataset and loader
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# Testing dataset and loader
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
# [6] Initialize network
model = NN(input_size=INPUT_SIZE, num_classes=NUM_CLASSES).to(device)

In [7]:
# [7] Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [10]:
# [8] Train Network
for epoch in range(NUM_EPOCHS):
    for batch_idx, (data, targets) in enumerate(train_loader):

        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # forward
        scores = model(data)
        loss =  criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step()

In [11]:
# [9] Check Accuracy on training and test to see how good is my model
def chackAccuracy(loader, model):
        if loader.dataset.train:
            print("Checking Accuracy on TRAIN data")
        else:
            print("Checking Accuracy on TEST data")
        num_correct = 0
        num_samples = 0
        model.eval()

        with torch.no_grad():
            for x,y in loader:
                x = x.to(device)
                y = y.to(device)
                x = x.reshape(x.shape[0], -1)

                scores = model(x)
                _, predictions = scores.max(1)
                num_correct += (predictions == y).sum()
                num_samples += predictions.size(0)
            print(f'Got {num_correct} / {num_samples} wich accuracy {float(num_correct)/float(num_samples)*100:.2f}')

        model.train()

chackAccuracy(train_loader, model)
chackAccuracy(test_loader, model)

Checking Accuracy on TRAIN data
Got 59483 / 60000 wich accuracy 99.14
Checking Accuracy on TEST data
Got 9740 / 10000 wich accuracy 97.40
